<img align="right" src="images/ninologo.png" width="150"/>
<img align="right" src="images/tf-small.png" width="125"/>
<img align="right" src="images/dans.png" width="150"/>

# Signs

Signs are the building blocks in the transcriptions.
They correspond to the individual "glyphs" on the tablet.

However, we have inserted a few *empty* signs, which we can leave out subsequently ...

We need a few extra modules.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections
from IPython.display import Markdown
from tf.app import use

In [3]:
A = use("uruk:clone", checkout="clone", hoist=globals())
# A = use('uruk', hoist=globals())

This is Text-Fabric 9.1.11
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

33 features found and 0 ignored


## Showing signs

The main characteristic of a sign is its **grapheme**.
Everything we do with signs, is complicated by the fact that signs can be *repeated*, and *augmented* with
*primes*, *variants*, *modifiers* and *flags*.

Before we go on, we call up our example tablet.

If you want to output multiple text items in an output cell, you have to `print()` it.

In [4]:
pNum = "P005381"
query = """
tablet catalogId=P005381
"""
results = A.search(query)
A.show(results, withNodes=True)

  0.00s 1 result


We navigate to the last sign in line 1 in column 2 on the obverse face:

In [5]:
case = A.nodeFromCase((pNum, "obverse:2", "1"))
sign1 = L.d(case, otype="sign")[-1]
print(sign1)

106611


That must be the right barcode.

We can retrieve the ATF transliteration:

In [6]:
print(A.atfFromSign(sign1))

GI4~a


Note that we get the ATF for a sign by means of `A.atfFromSign(node)`.
We get also the augments such as primes and modifiers and variant.
We get the flags if we say so by `flags=True`.

Take for example the first sign on line 3 in column 1 on the obverse face:

In [7]:
case = A.nodeFromCase((pNum, "obverse:1", "3"))
sign2 = L.d(case, otype="sign")[0]
print(sign2)
print(A.atfFromSign(sign2))
print(A.atfFromSign(sign2, flags=True))

106597
2(N04)
2(N04)#


Secondly, we want to get pointers to the locations of these signs in the corpus.

In [8]:
A.pretty(sign1, withNodes=True)
A.pretty(sign2, withNodes=True)

Click the links below `sign` and you are taken to the CDLI page for this tablet.

If we want to enlarge the sign, we can call it up with the lineart function.

In [9]:
A.lineart([sign1, sign2], width=200)

**N.B.**

For concepts that span one or more transliteration lines,
such as *tablet*, *face*, *column*, *line*, *case*, *comment*, you can get the source
material by requesting the feature **srcLn**, as we have seen before.

For *inline* concepts, such as *clusters*, *quads*, and *signs*,
there are functions in `A.`.

For signs we have:

* `atfFromSign(sign, flags=False)`

  Returns the ATF representation for a sign, including primes, repeats, variants, modifiers,
  and, optionally, flags.

The unaugmented transliteration of a single sign can be obtained from the feature **grapheme**:

In [10]:
print(F.grapheme.v(sign1))
print(F.grapheme.v(sign2))

GI4
N04


Let's pretty-print the line in which sign2 occurs:

In [11]:
A.pretty(L.u(sign2, otype="line")[0])

## Occurrences of a sign

Now we are using something we learned before: we want all signs with exactly
the grapheme `GU7`, regardless of augments or flags:

In [12]:
gu7s = F.grapheme.s("GU7")
len(gu7s)

314

Or, with a search template:

In [13]:
results = A.search(
    """
sign grapheme=GU7
"""
)

  0.11s 314 results


### With `table()` and `show()`
The simplest way to show the results is with `A.table()` for a compact tabular view, or with
`A.show()` with a full context view.

We show a tabular view of 3 occurrences, including node numbers.
The show view can be quite unwieldy, so we show a only 3 tablets.

In [14]:
A.table(results, withNodes=True, end=3)

n,p,sign
1,P001705 obverse:3:1,2456GU7
2,P001719 obverse:1:1,2660GU7
3,P001951 obverse:3:2,3883GU7


In [15]:
A.show(results, end=3, showGraphics=False)

### As a plain list

There are a few hundred occurrences, we show a bit more context for them, like we did before.
We show the full grapheme, with all its augments, **and flags**.
We also show the full source line.

In [16]:
for g in gu7s[0:10]:
    t = L.u(g, otype="tablet")[0]
    cl = A.lineFromNode(g)
    pNum = T.sectionFromNode(t)[0]
    gRep = A.atfFromSign(g, flags=True)

    line = F.srcLn.v(cl)

    print(f"{gRep:<7} {pNum} {line}")

GU7#?   P001705 1. 2(N47)# 6(N20)# 5(N05)#? 1(N42~a)# 1(N25)# 1(N28~c)#? , GU7#? [...] 
GU7#?   P001719 1. [...] 1(N14)# 2(N01)# , GU7#? [...] 
GU7     P001951 2. , GU7 
GU7#    P002002 1. , [...] EN~a# |SILA3~axSZE~a@t|# X [...] GU7# 
GU7#    P002035 2. , GU7# 
GU7#    P002062 1. [...] , [...] GU7# 
GU7     P002100 1. , [...] GU7 
GU7#    P002370 1. 1(N14) 1(N01) , GU7# [...] 
GU7     P002510 1. 1(N34) 1(N14) 1(N01) , GIR2~a GU7 
GU7     P002524 2. , GU7 


### As a linked table

We can make it more user friendly: we can link each occurrence to its page on CDLI, and
put everything in a Markdown table.

We have a function to generate the link: `A.cdli()`.

We build a markdown table.

We write a function for this, because we want to do it again.

First we use the function to write the first 10 to the screen,
and then to write the whole set to a directory on your file system.

In [17]:
def showSigns(signs, amount=None):

    markdown = """
    sign | tablet | line
    ---- | ------ | ----
    """.strip()

    for g in signs if amount is None else signs[0:amount]:
        t = L.u(g, otype="tablet")[0]
        cl = A.lineFromNode(g)
        gRep = A.atfFromSign(g, flags=True)
        line = F.srcLn.v(cl).replace("|", "&#124;")

        markdown += f"\n{gRep} | {A.cdli(t, asString=True)} | {line}"

    markdown += "\n"
    return markdown

In [18]:
Markdown(showSigns(gu7s, 3))

sign | tablet | line
    ---- | ------ | ----
GU7#? | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P001705" title="to CDLI main page for this item">P001705</a> | 1. 2(N47)# 6(N20)# 5(N05)#? 1(N42~a)# 1(N25)# 1(N28~c)#? , GU7#? [...] 
GU7#? | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P001719" title="to CDLI main page for this item">P001719</a> | 1. [...] 1(N14)# 2(N01)# , GU7#? [...] 
GU7 | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P001951" title="to CDLI main page for this item">P001951</a> | 2. , GU7 


A bit more please ...

In [19]:
Markdown(showSigns(gu7s, 10))

sign | tablet | line
    ---- | ------ | ----
GU7#? | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P001705" title="to CDLI main page for this item">P001705</a> | 1. 2(N47)# 6(N20)# 5(N05)#? 1(N42~a)# 1(N25)# 1(N28~c)#? , GU7#? [...] 
GU7#? | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P001719" title="to CDLI main page for this item">P001719</a> | 1. [...] 1(N14)# 2(N01)# , GU7#? [...] 
GU7 | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P001951" title="to CDLI main page for this item">P001951</a> | 2. , GU7 
GU7# | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P002002" title="to CDLI main page for this item">P002002</a> | 1. , [...] EN~a# &#124;SILA3~axSZE~a@t&#124;# X [...] GU7# 
GU7# | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P002035" title="to CDLI main page for this item">P002035</a> | 2. , GU7# 
GU7# | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P002062" title="to CDLI main page for this item">P002062</a> | 1. [...] , [...] GU7# 
GU7 | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P002100" title="to CDLI main page for this item">P002100</a> | 1. , [...] GU7 
GU7# | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P002370" title="to CDLI main page for this item">P002370</a> | 1. 1(N14) 1(N01) , GU7# [...] 
GU7 | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P002510" title="to CDLI main page for this item">P002510</a> | 1. 1(N34) 1(N14) 1(N01) , GIR2~a GU7 
GU7 | <a target="_blank" href="https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&amp;ObjectID=P002524" title="to CDLI main page for this item">P002524</a> | 2. , GU7 


### Everything to file

We give you the whole list, in a Markdown file,
on your local system.

In [20]:
with open(f"{A.tempDir}/gu7.md", "w") as fh:
    fh.write(showSigns(gu7s))

print(f"data written to file {A.tempDir}/gu7.md")

data written to file /Users/werk/github/Nino-cunei/uruk/_temp/gu7.md


<img src="images/atomMarkdown.png" width="600" align="right"/>

Have a look!

**Tip:** Open the file in [Atom](https://github.com/atom/atom).
Switch to preview by Ctr+Shift+M (in Atom).

Again, the tablet links are clickable, and bring you straight to CDLI.



## Frequency lists

We use a bit more power of Text-Fabric by generating frequency lists.

### Graphemes

We just studied the `GU7` grapheme a bit.
Suppose we want to get an overview of all graphemes?

There is a generic Text-Fabric function to give us that.
For each feature you can call up a frequency list of its values.

In [21]:
graphemes = F.grapheme.freqList()
len(graphemes)

632

We show the top-20:

In [22]:
graphemes[0:20]

(('…', 29413),
 ('N01', 21645),
 ('', 12440),
 ('X', 6870),
 ('N14', 5898),
 ('EN', 1950),
 ('N34', 1831),
 ('N57', 1826),
 ('SZE', 1334),
 ('GAL', 1180),
 ('DUG', 1084),
 ('U4', 1023),
 ('AN', 1020),
 ('PAP', 876),
 ('SAL', 876),
 ('NUN', 870),
 ('E2', 854),
 ('GI', 850),
 ('BA', 781),
 ('SANGA', 733))

**N.B.:**

* empty graphemes: `('', 12440),` These have been inserted by the conversion
  to Text-Fabric inside comments, in order to link them to the tablets.
* ellipsis graphemes: correspond to the `...` in ATF, usually within an uncertainty
  cluster `[...]`

### Augments

We can quickly get an overview of all kinds of augments: primes ,variants, modifiers, flags.

#### Prime

The prime is a feature with values: 2, 1 or 0.
The number indicates the number of primes.
Below you see how often that occurs.
Note that we count all primes here: on signs, case numbers and column numbers.

In [23]:
for (value, frequency) in F.prime.freqList():
    print(f"{frequency:>5} x {value}")

 5164 x 1
    1 x 2


#### Variant

The variant or *allograph* is what occurs after the grapheme and after the `~` symbol, which should be digits and/or
lowercase letters except the `x`.

Here is the frequency list of variant values.

In [24]:
for (value, frequency) in F.variant.freqList():
    print(f"{frequency:>5} x {value}")

23162 x a
 3994 x b
 1505 x c
 1308 x a1
  689 x b1
  188 x a2
  183 x d
  125 x b2
   85 x f
   72 x a3
   42 x e
   29 x c2
   22 x c1
   22 x c3
   14 x c5
   12 x a0
   12 x b3
   12 x d1
   12 x v
   11 x c4
    6 x a4
    6 x g
    5 x d2
    4 x d4
    4 x h
    2 x 3a
    2 x d3
    1 x h2


#### Modifier

The modifier is what occurs after the grapheme and after the `@` symbol
It consists of digits and/or
lowercase letters except the `x`.

Sometimes modifiers occur inside a repeat, then we have stored the modifier in the feature
*modifierInner*, as in

```
7(N34@f)
```

Here is the frequency list of *modifier* and *modifierInner* values.

In [25]:
for (value, frequency) in F.modifier.freqList():
    print(f"{frequency:>5} x {value}")

  634 x g
  262 x t
   35 x n
    6 x r
    4 x s
    1 x c
    1 x h


In [26]:
for (value, frequency) in F.modifierInner.freqList():
    print(f"{frequency:>5} x {value}")

   25 x f
    1 x r
    1 x v


### Full signs

We make a frequency list of all full signs, i.e. the grapheme including variant, modifier, and prime.
We show them as they appear in transcriptions.

We only deal with instances which are not contained in a quad.

This is no longer the frequency distribution of the values of a single feature,
so we have to do the coding ourselves.

In [27]:
fullGraphemes = collections.Counter()

for n in F.otype.s("sign"):
    grapheme = F.grapheme.v(n)
    if grapheme == "" or grapheme == "…" or grapheme == "X":
        continue
    fullGrapheme = A.atfFromSign(n)
    fullGraphemes[fullGrapheme] += 1

len(fullGraphemes)

1476

Or with a query:

In [28]:
query = """
sign type=ideograph|numeral
"""
fullGraphemesQ = {A.atfFromSign(r[0]) for r in A.search(query, silent=True)}
len(fullGraphemesQ)

1476

There! We have counted all incarnations of full graphemes, and there are 1476 distinct ones.

We show the top-20, sorted by frequency.

We specify a `key` function, that given an (value, amount) pair returns
(-amount, value).
This determines the order after sorting. Signs with a high value of amount come
before signs with a low value.

In [29]:
for (value, frequency) in sorted(
    fullGraphemes.items(),
    key=lambda x: (-x[1], x[0]),
)[0:20]:
    print(f"{frequency:>5} x {value}")

12983 x 1(N01)
 3080 x 2(N01)
 2584 x 1(N14)
 1830 x EN~a
 1598 x 3(N01)
 1357 x 2(N14)
 1294 x 5(N01)
 1294 x SZE~a
 1164 x GAL~a
 1117 x 4(N01)
 1022 x U4
 1020 x AN
  999 x 1(N34)
  876 x SAL
  851 x PAP~a
  849 x GI
  791 x 3(N14)
  789 x 1(N57)
  781 x BA
  719 x NUN~a


### Writing results to file

We also want to write the results to files in your `_temp` directory, within this repo.

`writeFreqs` writes distribution data of data items called `dataName`
to a file `fileName.txt`.
In fact, it writes two files:
* `fileName-alpha.txt`, ordered by data items
* `fileName-freq.txt`, ordered by frequency.

In [30]:
def writeFreqs(fileName, data, dataName):
    print(f"There are {len(data)} {dataName}s")

    for (sortName, sortKey) in (
        ("alpha", lambda x: (x[0], -x[1])),
        ("freq", lambda x: (-x[1], x[0])),
    ):
        with open(f"{A.tempDir}/{fileName}-{sortName}.txt", "w") as fh:
            for (item, freq) in sorted(data, key=sortKey):
                if item != "":
                    fh.write(f"{freq:>5} x {item}\n")

In [31]:
writeFreqs("grapheme-plain", F.grapheme.freqList(), "bare grapheme")

There are 632 bare graphemes


In [32]:
writeFreqs("grapheme-full", fullGraphemes.items(), "full grapheme")

There are 1476 full graphemes


Now have a look at your {{A.tempDir}} and you see two generated files:

* `graphemes-plain-alpha.txt` (sorted by grapheme)
* `graphemes-plain-freq.txt` (sorted by frequency)
* `graphemes-full-alpha.txt` (sorted by grapheme)
* `graphemes-full-freq.txt` (sorted by frequency)

# Next

[quads](quads.ipynb)

*Things never stay simple ...*

All chapters:
[start](start.ipynb)
[imagery](imagery.ipynb)
[steps](steps.ipynb)
[search](search.ipynb)
[calc](calc.ipynb)
**signs**
[quads](quads.ipynb)
[jumps](jumps.ipynb)
[cases](cases.ipynb)

---

CC-BY Dirk Roorda